# Using the mb_modelbase package on probabilistic models built in PyMC3 

Below is explained how `modelbase can be used to manipulate models built in PyMC3. This is necessary for displaying such models in the Lumen frontend. PyMC3 is a Python package for probabilistic programming that provides tools for describing and drawing inference from probabilistic Bayesian models.

Note that you need to install the mb_pymc3 package to be able to run this code.

## Setting up an example model
To illustrate the application, an example model is created first. The underlying data generating mechanism consists of a quantity mu which is drawn from a standard normal distribution, and a quantity x which is drawn from a normal distribution with mean mu. One hundred data points are drawn this way:


In [1]:
# Generate data for the example model
import numpy as np

np.random.seed(2)
size = 100
mu = np.random.normal(0, 1, size=size)
sigma = 1
X = np.random.normal(mu, sigma, size=size)

Assume now that the quantity x can be observed, but the quantity mu cannot be observed. Using PyMC3, we now fit a model to the observed data:

In [2]:
# Set up model with PyMC3
import pandas as pd
import pymc3 as pm

data = pd.DataFrame({'X': X})
basic_model = pm.Model()
with basic_model:
    sigma = 1
    mu = pm.Normal('mu', mu=0, sd=sigma)
    X = pm.Normal('X', mu=mu, sd=sigma, observed=data['X'])

## Perform operations on the model in mb_modelbase
mb_modelbase can now work with this model by creating an instance of the ProbabilisticPymc3Model class:

In [19]:
# Run operations on the model
from mb import modelbase as mbase
from mb.pymc3.pyMC3_model import ProbabilisticPymc3Model

In [20]:
modelname = 'pymc3_testcase_model'
m = ProbabilisticPymc3Model(modelname, basic_model)

In order to perform meaningful operations, the model has to be fitted. In the probabilistic context, this means that samples are created from the joint posterior disribution.

In [21]:
m.fit(data)

/home/luca_ph/venv/modelbase_venv/lib/python3.6/site-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
16:45:22.142 INFO :: Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
16:45:22.144 INFO :: Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
16:45:22.724 INFO :: Sequential sampling (1 chains in 1 job)
NUTS: [mu]
16:45:22.725 INFO :: NUTS: [mu]
Sampling 1 chain for 1_000 tune and 5_000 draw iterations (1_000 + 5_000 draws total) took 4 seconds.
16:45:26.258 INFO :: Sampling 1 chain for 1_000 tune and 5_000 draw iterations (1_000 + 5_000 draws total) took 4 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks
16:45:26.491 I

After that, is is possible to query probability density for specific points as well as calculating the point of highest probability density.

In [22]:
m.density([0,0])
m.aggregate_model(method='maximum')

[-0.04034574411281257, 0.004604064157903235]

It is also possible to marginalize and condition the model. Notice, that conditioning here only means that the of the domain variable is set. The model itself is not actually changed until the marginalize-method is called.

In [24]:
marginalized = m.copy().marginalize(remove='X')

In [25]:
condition = mbase.ConditionTuple('X', '>', 0)
conditioned = m.copy().condition(condition).marginalize(remove='X')

In [26]:
conditioned.aggregate_model(method='maximum')

[0.00954966244803028]

In [27]:
conditioned.density([0])

3.952328521428373